# Spatial Analysis

### In this notebook, we will be looking at the entire grid of our study area and creating some graphics to look at changes to surface temperature and upwelling

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
LandMask = xr.open_dataset(r'..\Data Samples\NetCDF\LandMask.nc').mask[0,:,:]

reject_outliers() is a function to remove any extreme outliers found in our data. We will be creating a "Day of Year" average, which is an average over the entire period of study on that day (January 1st, January 2nd, etc.). Then, we will take a snippet of each time frame (1985-2005, 2030-2050). We do this because we want to highlight the difference between present day (1985-2005) and the highest concentrations of greenhouse gasses under RCP 8.5. Finally, we snip down the data to just view data within the summer season (May 1st - September 30th), which studies have shown to have the highest Upwelling Intensities.

In [ ]:
def reject_outliers(data, m = 4.):
    d = np.abs(data - np.nanmedian(data))
    mdev = np.nanmedian(d)
    s = d/mdev if mdev else 0.
    return np.where(s<m, data, np.nan)

# Historical and Future Differences

The following function will create a historical and future intensity graphs for Ekman Transport, Ekman Pumping, or Surface Temperature. **Change "variable" in the first line of the block between "EkmanTransport", and "SurfaceTemp" to view the different variables** 

In [ ]:
# Set variable = 'EkmanTransport' or variable = 'SurfaceTemp'

variable = 'EkmanTransport'

variable_info={
    'EkmanTransport': {'verbose': 'Ekman Transport', 'label': 'm3/s/100m'},
    'SurfaceTemp': {'verbose':'Surface Temperature', 'label': 'C'}
}

BS = xr.open_dataset(r'..\Data Samples\NetCDF\{}\BS_DayofYear.nc'.format(variable))
RCP = xr.open_dataset(r'..\Data Samples\NetCDF\{}\RCP_DayofYear.nc'.format(variable))

BS_20 = getattr(BS, variable)
RCP_20 = getattr(RCP, variable)
BS_20.data = reject_outliers(BS_20.data, m=3.)
RCP_20.data = reject_outliers(RCP_20.data, m=3.)

BS_S_timeframe = BS_20[121:273]
RCP_S_timeframe = RCP_20[121:273]
    
BS_S_timeframe.load()
RCP_S_timeframe.load()

lat = BS.lat.data
lon = BS.lon.data

vmin= min(BS_S_timeframe.mean(dim='dayofyear').min(), RCP_S_timeframe.mean(dim='dayofyear').min())
vmax = max(BS_S_timeframe.mean(dim='dayofyear').max(), RCP_S_timeframe.mean(dim='dayofyear').max())

figsize = [10,10]
projection= crs.LambertConformal()
geographic_extent = [lon[0,0], lon[-1,-1], lat[0, 0], lat[-1,-1]]
fontsize=16
colorbar_rect = [0.8,0.20,0.03,0.6]
cmap = 'jet'
label = variable_info[variable]['label']
verbose_name = variable_info[variable]['verbose']

fig1 = plt.figure(figsize=figsize)
ax1 = fig1.add_subplot(1,2,1,projection=projection)
ax1.set_title('{}: Historical'.format(verbose_name), fontsize=fontsize)
ax1.set_extent(geographic_extent, crs.Geodetic())
plot = ax1.pcolormesh(
    lon, 
    lat, 
    BS_S_timeframe.mean(dim='dayofyear').where(LandMask==0),
    transform=crs.PlateCarree(), 
    cmap=cmap, 
    vmin=vmin, 
    vmax=vmax
)
bar_ax = fig1.add_axes(rect=colorbar_rect) 
colorbar = fig1.colorbar(plot, cax=bar_ax)
colorbar.set_label(label, fontsize=fontsize)

fig2 = plt.figure(figsize=figsize)
ax2 = fig2.add_subplot(1,2,1,projection=projection)
ax2.set_title('{}: Future'.format(verbose_name), fontsize=fontsize)
ax2.set_extent(geographic_extent, crs.Geodetic())
plot = ax2.pcolormesh(
    lon, 
    lat, 
    RCP_S_timeframe.mean(dim='dayofyear').where(LandMask==0),
    transform=crs.PlateCarree(), 
    cmap=cmap, 
    vmin=vmin, 
    vmax=vmax
)
bar_ax = fig2.add_axes(rect=colorbar_rect) 
colorbar = fig2.colorbar(plot, cax=bar_ax)
colorbar.set_label(label, fontsize=fontsize)

fig3minmax={'EkmanTransport':{'vmin':-6,'vmax':0}, 'EkmanPumping':{'vmin':-.00002, 'vmax':.00002}, 'SurfaceTemp':{'vmin':.8,'vmax':1.3}}
fig3 = plt.figure(figsize=figsize)
ax3 = fig3.add_subplot(1,2,1,projection=projection)
ax3.set_title('{}: Duration'.format(verbose_name), fontsize=16)
ax3.set_extent(geographic_extent, crs.Geodetic())
plot = ax3.pcolormesh(
    lon, 
    lat, 
    (RCP_S_timeframe.mean(dim='dayofyear') - BS_S_timeframe.mean(dim='dayofyear')).where(LandMask==0),
    transform=crs.PlateCarree(),
    cmap=cmap,
    vmin=fig3minmax[variable]['vmin'],
    vmax=fig3minmax[variable]['vmax']
)
bar_ax = fig3.add_axes(rect=colorbar_rect) 
colorbar = fig3.colorbar(plot, cax=bar_ax)
colorbar.set_label(label, fontsize=fontsize)

for ax in [ax1,ax2, ax3]:
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND)
    ax.gridlines()   


From these graphs, we can see a decrease in upwelling in the mid latitudes in the future, and an increase in Surface Temperature in the Future.